In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

import warnings
warnings.filterwarnings('ignore')

sns.set()
plt.style.use('ggplot')

In [ ]:
df = pd.read_csv("breast-cancer.csv")

In [ ]:
df.head()

In [ ]:
# DATA PREPROCESSING
df.diagnosis.unique()

In [ ]:
#M-> Malignint
#B-> Benign

In [ ]:
# Supervised-> target
# Unsupervised
df.describe()

In [ ]:
df.info()

In [ ]:
#misisng value
df.isnull().sum()

In [ ]:
msno.bar(df, color="yellow")

In [ ]:
# There is no missing value in the dataset
# so we don't need to handle out

In [ ]:
df['diagnosis'] = df['diagnosis'].apply(lambda val:1 if val=='M' else 0)

In [ ]:
plt.hist(df['diagnosis'])
plt.title('Diagnosis(M=1, B=0)')
plt.show()

In [ ]:
#EDA

In [ ]:
# each 5 row its having 6 columns
# density graph

plt.figure(figure=(20,15))
plotnumber=1
for column in df:
    if plotnumber <=30:
        ax = plt.subplot(6,7, plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column)
    plotnumber+=1

plt.tight_layout()
plt.show()

In [ ]:
df.corr()

In [ ]:
#heatmap
plt.figure(figsize=(20,12))
corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, linewidths=1, annot=True, fmt = ".2f")
plt.show()

In [ ]:
# highly correlated feature
# multicollinearity

In [ ]:
df.drop('id', axis=1, inplace=True)

In [ ]:
#feature column
corr_matrix = df.corr().abs()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
tri_df = corr_matrix.mask(mask)
#features having value greater than 0.92 drop them
to_drop = [x for x in tri_df.columns if any(tri_df[x]>0.92)]

df = df.drop(to_drop, axis=1)

print(df.shape[1])

In [ ]:
df.head()

In [ ]:
# 32 feature reduce it to 23

In [ ]:
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, random_state=0)

In [ ]:
#Scaling data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
# Apply Machine Learninig Algorithm

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

In [ ]:
y_pred = log_reg.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_train, log_reg.predict(X_train)))
log_reg_acc = accuracy_score(y_test, log_reg.predict(X_test))
print(log_reg_acc)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
y_pred = log_reg.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_train, knn.predict(X_train)))
knn_acc = accuracy_score(y_test, knn.predict(X_test))
print(knn_acc)
y_pred = knn.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# SVM
# Hyperparameter tuning
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svc = SVC(probability=True)

parameters = {
    'gamma' : [0.0001, 0.001, 0.01, 0.1],
    'C' : [0.01, 0.05, 0.5, 0.1, 1, 10, 15, 20]
}
grid_search = GridSearchCV(svc, parameters)
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
svc = SVC(C=15, gamma=0.01, probability=True)
svc.fit(X_train, y_train)

In [ ]:
y_pred = svc.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_train, svc.predict(X_train)))
svc_acc = accuracy_score(y_test, svc.predict(X_test))
print(svc_acc)
y_pred = svc.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
#Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

parameters = {
    'criterion':['gini','entropy'],
    'max_depth': range(2,32,1),
    'min_samples_leaf':range(1,10,1),
    'min_samples_split':range(2,10,1),
    'splitter':['best','random']
}

grid_search_dt = GridSearchCV(dtc, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search_dt.fit(X_train, y_train)

In [ ]:
grid_search_dt.best_params_

In [ ]:
grid_search_dt.best_score_

In [ ]:
dtc = DecisionTreeClassifier(criterion='entropy', max_depth=15, min_samples_leaf = 4, min_samples_split = 5, splitter='random')

In [ ]:
dtc.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_train, dtc.predict(X_train)))
dtc_acc = accuracy_score(y_test, dtc.predict(X_test))
print(dtc_acc)
y_pred = dtc.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rand_clf = RandomForestClassifier(criterion = 'entropy', max_depth = 10, max_features = 0.5, min_samples_leaf=2, min_samples_split = 3, n_estimators = 130)
rand_clf.fit(X_train, y_train)

In [ ]:
y_pred = rand_clf.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_train, rand_clf.predict(X_train)))
rand_clf_acc = accuracy_score(y_test, rand_clf.predict(X_test))
print(rand_clf_acc)
y_pred = rand_clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

# Initialize the Gradient Boosting Classifier
gbc = GradientBoostingClassifier()

# Define the parameter grid
parameters = {
    'loss': ['deviance', 'exponential'],
    'learning_rate': [0.001, 0.1],
    'n_estimators': [100, 150, 180]
}

# Set up GridSearchCV with corrected parameters
grid_search_gbc = GridSearchCV(gbc, parameters, cv=2, n_jobs=-1, verbose=1)

# Fit the grid search to the data (Assuming X_train and y_train are already defined)
grid_search_gbc.fit(X_train, y_train)


In [ ]:
grid_search_gbc.best_params_

In [ ]:
grid_search_gbc.best_score_

In [ ]:
gbc = GradientBoostingClassifier(learning_rate = 0.1, loss = 'exponential', n_estimators = 180)
gbc.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_train, gbc.predict(X_train)))
gbc_acc = accuracy_score(y_test, gbc.predict(X_test))
print(gbc_acc)
y_pred = gbc.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(objective = 'binary:logistic', learning_rate = 0.01, max_depth = 5, n_estimators = 180)
xgb.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_train, xgb.predict(X_train)))
xgb_acc = accuracy_score(y_test, xgb.predict(X_test))
print(xgb_acc)
y_pred = xgb.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
models = pd.DataFrame({
    'Model': ['Logistic Regression', 'KNN', 'SVC',  'Decision Tree Classifier', 'Random Forest Classifier', 'Gradient Boosting Classifier', 'XGB Classifier'],
    'Score': [100*round(log_reg_acc,4), 100*round(knn_acc,4), 100*round(svc_acc,4), 100*round(dtc_acc,4), 100*round(rand_clf_acc,4), 100*round(gbc_acc,4), 100*round(xgb_acc,4)]
})
models.sort_values( by = 'Score', ascending = False)
   

In [ ]:
import pickle
model = svc
pickle.dump(model, open("breast_cancer.pkl","wb"))

In [ ]:
from sklearn import metrics
plt.figure(figsize=(8,5))
models = [
{
     'label': 'LR',
     'model': log_reg,
},
{
     'label': 'DT',
     'model': dtc,
},    
{
     'label': 'SVM',
     'model': svc,
},
{
     'label': 'KNN',
     'model': knn,
},
{
     'label': 'XGBoost',
     'model': xgb,
},
{
     'label': 'RF',
     'model': rand_clf,
},
{
     'label': 'GBDT',
     'model': gbc,
}
]
for m in models:
    model = m['model']
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    fprl, tprl, thresholds = metrics.roc_curve(y_test, model.predict_proba(X_test)[:,1])
    auc  = metrics.roc_auc_score(y_test,model.predict(X_test))
    plt.plot(fprl, tprl, label = '%s - ROC (area = %0.2f)' % (m['label'], auc))

plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1 - Specificity (False Positive Rate)', fontsize = 12)
plt.ylabel('Sensitivity (True Positive Rate)', fontsize = 12)
plt.title('ROC - Breast Cancer Prediction', fontsize = 12)
plt.legend(loc="lower right", fontsize = 12)
plt.savefig("roc_breast_cancer.jpeg", format = 'jpeg',  dpi = 400, bbox_inches='tight')
plt.show()

In [ ]:
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
models = [
{
     'label': 'LR',
     'model': log_reg,
},
{
     'label': 'DT',
     'model': dtc,
},    
{
     'label': 'SVM',
     'model': svc,
},
{
     'label': 'KNN',
     'model': knn,
},
{
     'label': 'XGBoost',
     'model': xgb,
},
{
     'label': 'RF',
     'model': rand_clf,
},
{
     'label': 'GBDT',
     'model': gbc,
}
]
means_roc = []
means_accuracy =  [100*round(log_reg_acc,4), 100*round(knn_acc,4), 100*round(svc_acc,4), 100*round(dtc_acc,4),
          100*round(rand_clf_acc,4), 100*round(gbc_acc,4), 100*round(xgb_acc,4)]

for m in models:
    model = m['model']
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    fprl, tprl, thresholds = metrics.roc_curve(y_test, model.predict_proba(X_test)[:,1])
    auc  = metrics.roc_auc_score(y_test,model.predict(X_test))
    auc = 100*round(auc, 4)
    means_roc.append(auc)

print(means_accuracy)
print(means_roc)

#data to plot
n_groups = 7
means_accuracy = tuple(means_accuracy)
means_roc = tuple(means_roc)

# Create plot
fig, ax = plt.subplots(figsize = (8,5))
index = np.arange(n_groups)
bar_width = 0.35
opacity = 0.8

rects1 = plt.bar(index, means_accuracy, bar_width,
                 alpha = opacity,
color = 'mediumpurple',
label='Accuracy (%)')

rects2 = plt.bar(index + bar_width, means_roc, bar_width,
                 alpha = opacity,
color = 'rebeccapurple',
label='ROC (%)')

plt.xlim([-1, 8])
plt.ylim([70, 104])

plt.title('performance Evaluation - Breast Cancer Prediction', fontsize = 12)
plt.xticks(index, (' LR', ' DT',' SVM', ' KNN', 'XGBoost', 'RF', 'GBDT'), rotation=40)
plt.legend(loc="upper right", fontsize = 10)
plt.savefig("PE_breast-cancer.jpeg", format = 'jpeg', dpi = 400, bbox_inches='tight')
plt.show()